# Chapter 9 - Planning and Learning



### Exercise 9.1

The nonplanning method looks particularly poor in Figure 9.6 because it is a one-step method; a method using eligibility traces would do better. Do you think an eligibility trace method could do as well as the Dyna method? Explain why or why not.

__Answer__: I expect eligibility traces methods to perform generally worse compared to Dyna-Q in this deterministic task (the Dyna Maze), but could be on par under certain conditions. In particular, there are three factors we should consider: the number of backup updates, the magnitude of the updates (controlled by $\alpha$ and $\lambda$) and the coverage of state-action space. Let's take them one by one.

At each step taken in the real environment, Dyna-Q makes $N$ updates via planning, so in total $N T$ updates of the action values table $Q(\cdot, \cdot)$ per episode. Eligibility traces update all previous steps of the episode at each step, so roughly $1 + 2 + ... + T = \dfrac {T(T-1)} {2}$ updates. So for long episodes, when $T > 2N$, eligibility traces seem to have the advantage of making more updates to $Q$ than Dyna.

However, in terms of the magnitude of those updates, for this task where the only non-zero reward is received only by reaching the goal $G$, eligibility traces perform exponentially smaller updates as they move backwards in time, as controlled by the $\lambda$ parameter. So the true values would be slow in propagating back to early states for a small $\lambda$. Compare Figure 7.12 with Figure 9.6. But if $\lambda \approx 1$, the updates should be exactly the same as the planning updates from Dyna.

Finally, we need to keep in mind that the updates in Dyna are randomly chosen from all previous transitions, which means that coverage of the state-action space could be broader early on in Dyna, compared to eligibility traces, which can only update previous steps from the current episode. However, Dyna can also waste large numbers of update steps later on, by randomly choosing them far from the optimal path, whereas eligibility traces would only update states that are closer to the optimal path (due to policy improvement over time).

Given all these reasons, one would expect Dyna methods to perform better than eligibility traces, especially early on, due to faster convergence. But under certain conditions (long episodes and $\lambda$ close to $1$) eligibility traces should be able to match its performance.